In [1]:
from __future__ import division

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

from model import *
from utils import *

In [ ]:
blocks = parse_cfg('./cfg/model.cfg')
create_modules(blocks)[1]

## Define Network

## Test Foward Pass

In [44]:
net = Net("cfg/model.cfg")
input = test_forward_pass()
pred = net(input, torch.cuda.is_available())
pred

tensor([[[2.2937e+01, 1.5843e+01, 9.5361e+01,  ..., 4.2360e-01,
          4.2999e-01, 4.6617e-01],
         [2.0630e+01, 2.0300e+01, 3.1507e+02,  ..., 4.3634e-01,
          4.5117e-01, 5.2884e-01],
         [2.2486e+01, 2.4395e+01, 3.7954e+02,  ..., 5.4638e-01,
          5.4957e-01, 5.7353e-01],
         ...,
         [5.6665e+02, 5.6660e+02, 8.9677e+00,  ..., 4.8677e-01,
          3.8770e-01, 4.3729e-01],
         [5.6562e+02, 5.6691e+02, 2.2953e+01,  ..., 5.1666e-01,
          5.3433e-01, 5.6511e-01],
         [5.6569e+02, 5.6543e+02, 4.0473e+01,  ..., 5.2003e-01,
          4.6440e-01, 6.1079e-01]]])

In [43]:
pred.shape

torch.Size([1, 10647, 85])

size: batch_size x n_bboxes x (4 bbox attrs + 1 object score + 80 class scores)